**Project Overview**

The core idea is to develop a system that can interpret and respond to user queries related to mental health. This will be achieved by fine-tuning the Mixtral 8x7B model, a Mixture of Experts (MoE) model known for its efficient handling of diverse tasks and large-scale data, on a targeted dataset from Hugging Face that includes mental health discussions or related content.

**Dataset**: Utilizing a dataset specifically related to mental health, likely comprising textual data that includes questions, discussions, and information about mental health issues. This dataset is pivotal for training the model to understand and generate relevant responses within this context.

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('marmikpandya/mental-health')

# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(dataset['train'])

In [ ]:
df.columns

In [ ]:
df.output

In [ ]:
df.instruction

In [ ]:
# Display basic information
print(df.info())
print(df.head())

In [ ]:
df = df.dropna()  # Or use df.fillna(method='ffill') to fill missing values


In [ ]:
def preprocess_data(dataset):

    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(dataset['train'])

    # Example of removing irrelevant features
    relevant_columns = ['input']  # Assuming 'text' is the relevant column
    df = df[relevant_columns]

    # Handling missing data
    df = df.dropna()

    return df

# Preprocess the dataset
df_clean = preprocess_data(dataset)
print("Data preprocessing complete.")

In [ ]:
from transformers import AutoTokenizer

In [ ]:
!pip install transformers


In [ ]:
from transformers import AutoTokenizer, BertTokenizer,DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelForCausalLM

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

# Load the model with specific settings for 4-bit quantization and device mapping
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    # device_map="auto"3 re
)

print("Model and tokenizer have been loaded with the specified configurations.")

In [ ]:
!conda install cudatoolkit=11

In [ ]:
# Adding a soft prompt to the tokenizer
soft_prompt = "The mental state described is: "
tokenizer.pad_token = tokenizer.eos_token  # Ensuring padding is handled

def add_soft_prompt(texts):
    return [soft_prompt + text for text in texts]

# Update dataset texts
df_clean['text'] = add_soft_prompt(df_clean['text'].tolist())
print("Soft prompts added to texts.")

In [ ]:
# Fine-Tuning the Model

from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
    push_to_hub=False,               # Whether to push model to the hub
    fp16=True,                       # Use mixed precision training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_clean,
    eval_dataset=df_clean,  # Ideally, you should have a separate validation set
)

# Train the model
trainer.train()
print("Model fine-tuning complete.")

In [ ]:
#After training, evaluate your model to see how well it performs:

eval_results = trainer.evaluate(
print("Evaluation results:", eval_results)


# Save the model for deployment
model.save_pretrained('./mental_health_chatbot_model')



In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the trained model and tokenizer
model_path = './mental_health_chatbot_model'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained('google/mixtral-8x7b')  # Assuming you used this tokenizer

# Make sure the model is in evaluation mode
model.eval()


Step 2: Create a Function to Generate Responses
You will need a function that takes user input, tokenizes it, passes it through the model, and then decodes the model’s output into a human-readable response.

In [ ]:
import torch

def generate_response(user_input):
    # Tokenize the user input
    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move input tensors to the same device as model
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    # Generate output logits from the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities (softmax)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Assuming a binary classification (e.g., 0: No issue, 1: Issue detected)
    response_label = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0, response_label].item()

    # Define responses based on the detected label
    if response_label == 1:
        return f"There seems to be a concern. I highly recommend seeking professional help. (Confidence: {confidence:.2f})"
    else:
        return f"Everything seems normal. If you feel this isn't right, please consult a professional. (Confidence: {confidence:.2f})"


setting up a Simple Command-Line Interface
This allows for real-time interaction with the chatbot.

In [ ]:
def main():
    print("Hello! I'm your Mental Health Assistant. How can I assist you today?\n")
    try:
        while True:
            user_input = input("You: ")
            if user_input.lower() in ['exit', 'quit']:
                print("Goodbye!")
                break
            response = generate_response(user_input)
            print("Bot:", response)
    except KeyboardInterrupt:
        print("\nGoodbye!")
        
if __name__ == "__main__":
    main()